# This is my notebook

# Setup

In [ ]:
# Setup
import os
import subprocess

# For traversing files
from pathlib import Path

# For logging
import logging
import sys

# For saving notebook
from IPython.display import display, Javascript

# For other amazon shortcuts
import json
import tempfile
import shlex
import re

## Logger

In [ ]:
# Create logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create STDERR handler
handler = logging.StreamHandler(sys.stderr)
# Uncomment to set logLevel to debug
# ch.setLevel(logging.DEBUG)

# Create formatter and add it to the handler
formatter = logging.Formatter('%(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

# Set STDERR handler as the only handler 
logger.handlers = [handler]

## AWS Shortcuts

In [ ]:
# AWS short-cuts
def get_aws_attribute(attribute_name):
    """
    Available attributes
    "accountId", "architecture", "availabilityZone", "billingProducts", 
    "devpayProductCodes", "marketplaceProductCodes", "imageId", "instanceId", 
    "instanceType", "kernelId", "pendingTime", 
    "privateIp", "ramdiskId", "region", "version",
    :param attribute_name: 
    :return: 
    """
    
    # Use subprocess to get an attribute from 169.254.169.254
    document_url = "http://169.254.169.254/latest/dynamic/instance-identity/document"
    get_aws_attribute_command = ["curl", "-s", document_url]
    get_aws_attribute_proc = subprocess.run(get_aws_attribute_command, capture_output=True)

    get_aws_attribute_proc__stdout = get_aws_attribute_proc.stdout.decode()
    get_aws_attribute_proc__stderr = get_aws_attribute_proc.stderr.decode()
    
    if not get_aws_attribute_proc.returncode == 0:
        logger.error("Got non-zero exit code running curl command to document")
        logger.error("Stdout: {}".format(get_aws_attribute_proc__stdout))
        logger.error("Stderr: {}".format(get_aws_attribute_proc__stderr))
        return None
    
    # Successful curl command
    get_aws_attribute_proc__stdout_as_json = json.loads(get_aws_attribute_proc__stdout.strip())
    
    # Ensure the attribute is actually there    
    if attribute_name not in get_aws_attribute_proc__stdout_as_json.keys():
        logger.error("Could not find {} attribute from {}".format(attribute_name, document_url))
        logger.error("Returning none")
        return None
    else:
        return get_aws_attribute_proc__stdout_as_json[attribute_name]

def get_this_instance_id():
    """
    Use get_aws_attribute function to get this instance Id
    :return: 
    """
    instance_id = get_aws_attribute("instanceId")
    return instance_id

def get_this_region():
    """
    Use get_aws_attribute function to get this region
    :return: 
    """
    availability_zone = get_aws_attribute("availabilityZone")
    
    re_compile = re.compile("^(\S+)-(\S+)-(\d+)(?:\S+)$")
    
    re_match = re.match(re_compile, availability_zone)
    
    if re_match is not None:
        region = '-'.join([re_match.group(1), re_match.group(2), re_match.group(3)])
    else:
        logger.warning("Couldn't get a match to region")
        region = None
        
    return region

def stop_it_this_instance():
    """
    Stop the instance from inside the notebook.
    Use the at command to schedule this for one minute in advance
    Useful if you've just run a long bit of code and want to let the instance rest afterwards
    :return: 
    """
    
    # Stop command
    stop_command = ['sudo', 'halt']
    
    # Write stop command to tmp file
    stop_command_fobj = tempfile.NamedTemporaryFile(delete=False)
    with open(stop_command_fobj.name, 'w') as stop_command_fh:
        # Add shebang
        stop_command_fh.write("#!/bin/bash\n")
        stop_command_fh.write(" ".join([shlex.quote(command_arg) 
                                        for command_arg in stop_command]))
        stop_command_fh.write("\n")
    
    # Run proc, via at command, program to shut down in one minute
    at_command = ["at", "now", "+", "2", "minute", "-f", stop_command_fobj.name]
    
    at_proc = subprocess.run(at_command, capture_output=True)
    
    if not at_proc.returncode == 0:
        logger.error("At comment returned with non-zero exit value")
        logger.error(at_proc.stdout.decode())
        logger.error(at_proc.stderr.decode())
    else:
        logger.info("Shutdown successfully activated for somewhere between 60 and 120 seconds time")
    
    return at_proc.returncode

# Content goes here

In [13]:
%%javascript
IPython.notebook.save_notebook()

<IPython.core.display.Javascript object>

In [ ]:
# Auto-shutdown - if on a non spot-instance
stop_it_this_instance()